In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Load Dataset and Analyze

In [25]:
df =pd.read_csv('spam_final.csv',header=None,names=["label","message"],engine="python")

In [26]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [27]:
df.isnull().sum()

label      0
message    0
dtype: int64

In [28]:
df.shape

(5572, 2)

In [29]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

### Balance Dataset

In [32]:
ham=df[df['label']=="ham"]
ham.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
6,ham,Even my brother is not like to speak with me. ...


In [33]:
spam=df[df['label']=="spam"]
spam.head()

,label,message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [34]:
ham.shape

(4825, 2)

In [35]:
spam.shape

(747, 2)

In [37]:
ham = ham.sample(spam.shape[0])

In [38]:
ham.shape, spam.shape

((747, 2), (747, 2))

In [40]:
data = ham.append(spam, ignore_index=True)

In [41]:
data.sample()

,label,message
62,ham,I wish that I was with you. Holding you tightl...
